<a href="https://colab.research.google.com/github/mlKwon/crawling/blob/main/amazon_shopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [48.6 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,326 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os
import gc
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + user_agent)
options.add_argument('--start-maximized')
options.add_argument('incognito')
service = Service()

try:
    driver = webdriver.Chrome(service=service, options=options)
except Exception as e:
    print(e)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob

os.chdir('drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


Numbuzin

In [4]:
## Numbuzin

In [ ]:
driver.quit()
service = Service()
driver = webdriver.Chrome(service=service, options=options)
url = "https://www.amazon.com/stores/page/BDB9CC0A-DA48-41E3-ACD8-B0255744997D?ingress=2&visitId=a0a7c6d0-3822-4620-98ac-077900f58183&ref_=ast_bln" # Amazon 넘버즈인 브랜드관
driver.get(url)
driver.implicitly_wait(1)

temp = driver.find_elements(By.CSS_SELECTOR, 'span.Navigation__linkText__LoQD4.Navigation__en-US__YUfTQ') # Category

temp = temp[2:-1]
temp_text = [tt.text for tt in temp] # Category name
temp_link = driver.find_elements(By.CSS_SELECTOR, 'li.Navigation__navItem__bakjf a')[2:-1]
temp_link = [link.get_attribute('href') for link in temp_link] # Category link

link_prd_len = [7,10,5,3,1,2,5] # number of products for each category : 23.12.31

review_all = []
global_i = 0
b_stopped = False

for i, link in enumerate(temp_link):
  # if i<1: continue
  print('')
  print(temp_text[i])

  driver.get(link)

  if temp_text[i] not in ['CLEANSER',"SUNSCREEN",'TONE UP']:
    selector = 'div.ProductGridItem__imageContainer__Ts5N_.default a'
  else:
    # selector = 'a.ProductShowcase__title__SBCBw'
    selector = 'a.ProductShowcase__rating__Wi83c'

  product_link = [dd.get_attribute('href') for dd in driver.find_elements(By.CSS_SELECTOR, selector)] # product link
  while len(product_link) < link_prd_len[i]:
    product_link = [dd.get_attribute('href') for dd in driver.find_elements(By.CSS_SELECTOR, selector)]

  print(f'link 수:{len(product_link)}')

  for j, prd_link in enumerate(product_link):
    review_dict = {'prd_type':'0','prd_name':'a','title':'d','star':'c','review':'e','date':'b','country':'cc'}

    driver.get(prd_link)
    prd_name = driver.find_elements(By.CSS_SELECTOR, '#productTitle')
    prd_name = prd_name[len(prd_name)-1].text
    print(f'\n{j+1}=> {prd_name}')

    # go to review
    while True:
      try:
        if b_stopped:
          print('restart')
          driver.get(prd_link)
          driver.implicitly_wait(1)
          b_stopped = False

        try:
          review_link = driver.find_elements(By.CSS_SELECTOR,'div.a-row.a-spacing-medium a')[1].get_attribute('href') # US review
          b_skip_title = True
        except:
          review_link = driver.find_elements(By.CSS_SELECTOR,'a[data-hook="see-all-reviews-link-foot"]')[0].get_attribute('href') # non-US review
          b_skip_title = False

        driver.get(review_link)

        rating_info_element = driver.find_element(By.CSS_SELECTOR, 'div[data-hook="cr-filter-info-review-rating-count"]')
        rating_info_text = rating_info_element.text.strip()
        # 정규표현식을 사용하여 숫자 추출
        match = re.search(r'(\d+) total ratings, (\d+) with reviews', rating_info_text)

        if match:
            total_ratings = match.group(1)
            with_reviews = match.group(2)

        print(f'number of reviews: {with_reviews}')

        n_review = int(with_reviews)
        if n_review % 10 > 0:
          n_rep = n_review//10 + 1
        else:
          n_rep = n_review//10

        print('pages: ', end='')

        review_middle_all = []
        for rep in range(n_rep):
          time.sleep(.5)

          if b_skip_title: review_title = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-title"]')[2:]]
          else: review_title = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-title"]')]
          review_date = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,"span[data-hook='review-date']")]
          review_country = [re.search(r'Reviewed in (.*?) on (.*)', tt).group(1) for tt in review_date]
          review_date = [re.search(r'Reviewed in (.*?) on (.*)', tt).group(2) for tt in review_date]
          review_star = [tt.get_attribute("textContent") for tt in driver.find_elements(By.CSS_SELECTOR,"i[data-hook='review-star-rating'] span.a-icon-alt")]
          if len(review_star) < len(review_date): # other language
            review_star = review_star + [tt.get_attribute("textContent") for tt in driver.find_elements(By.CSS_SELECTOR,"i[data-hook='cmps-review-star-rating'] span.a-icon-alt")]
          review_star = [re.search(r'(\d+\.\d+) out of \d+ stars', tt).group(1) for tt in review_star]

          review_body = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-body"]')]
          # print(f'리뷰갯수: {len(review_title),len(review_date), len(review_star), len(review_body)}')

          print(rep+1, end=" ")
          # review_star = driver.execute_script("return document.querySelector('i[data-hook=\"review-star-rating\"] span.a-icon-alt').textContent")
          # print(f'Title: {review_title[0]}')
          # print(review_date[0])
          # print(review_star[0])
          # print(review_body[0])

          for k in range(len(review_body)): # for each review
            review_dict['prd_type'] = temp_text[i]
            review_dict['prd_name'] = prd_name
            review_dict['title'] = review_title[k]
            review_dict['star'] = review_star[k]
            review_dict['review'] = review_body[k]
            review_dict['date'] = review_date[k]
            review_dict['country'] = review_country[k]
            review_middle_all = review_middle_all + [pd.DataFrame([review_dict])]

          driver.find_elements(By.CSS_SELECTOR,'li.a-last')[0].click() # next button

        review_all = review_all + review_middle_all # for each product

        print('')

        break

      except Exception as e:
        # 해당 제품에서 error 발생시, while문에서 restart
        print(f"Error: {e}")
        # account login 요구 시, 해당 부분 실행
        try:
          acnt = pd.read_csv('acnt_inform.csv') # your account information
          email_input = driver.find_element(By.ID, "ap_email")
          email_value = acnt.email.iloc[0]
          email_input.send_keys(email_value)

          continue_button = driver.find_element(By.ID, "continue")
          continue_button.click()

          password_input = driver.find_element(By.ID, "ap_password")
          password_value = acnt.password.iloc[0]
          password_input.send_keys(password_value)

          print('login complete')
        except:
          # 다른 에러의 경우,
          print('not login error')

        b_stopped = True
        global_i = i
        global_j = j

driver.quit()


TONER
link 수:7

1=> numbuzin No.1 Pure-Full Calming Herb Toner | Acne-Prone Skin, Centella Asiatica, Deep Calming (10.14 Fl Oz)
number of reviews: 44
pages: 1 2 3 4 5 

2=> numbuzin No.3 Super Glowing Essence Toner | Fermented Ingredients, Niacinamide, Galactomyces, glowy Skin Radiance | Korean Skin Care for Face (6.76 Fl Oz)
number of reviews: 636
pages: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 

3=> numbuzin No.4 Hydrating Glow Mineral Toner | Thermal Water, Hyaluronic Acid, Hydration, Makeup Boosting, 6.76 fl.oz
number of reviews: 32
pages: 1 2 3 4 

4=> numbuzin No.1 Centella Re-Leaf Green Toner Pad | Vegan Pad, Double-Sided, Herb extracts, Calming, 70 Pads, 6.42 fl.oz
number of reviews: 41
pages: 1 2 3 4 5 

5=> numbuzin No.3 Radiance Glowing Jumbo Essence Pad | Vegan Pad, Unbleached, Biodegradable, Niacinamide, Clear Glass Skin | Korean Sk

In [ ]:
import os, glob

review_df = pd.concat(review_all).drop_duplicates()
review_df.groupby(['country'])['date'].count()

In [ ]:
review_df.to_csv('../numbuzin/amazon_numbuzin_review.csv',index=None)

Anua

In [13]:
review_all_anua = []

In [15]:
driver.quit()
service = Service()
driver = webdriver.Chrome(service=service, options=options)
url = "https://www.amazon.com/stores/page/D1BEFB29-D065-44FC-81A2-E5A8526E1651?store_ref=storeRecs_sa__x&amp=&ref_=nb_sb_ss_w_stores-v2-t2_k0_11_4_1_1&crid=3OBTYQF3XQ97J&amp=&sprefix=anua&amp=&redirect_store_id=D1BEFB29-D065-44FC-81A2-E5A8526E1651" # Amazon Anua 브랜드관
driver.get(url)
driver.implicitly_wait(1)

temp = driver.find_elements(By.CSS_SELECTOR, 'span.Navigation__linkText__LoQD4.Navigation__en-US__YUfTQ') # Category

temp = temp[2:-1]
temp_text = [tt.text for tt in temp] # Category name
temp_link = driver.find_elements(By.CSS_SELECTOR, 'li.Navigation__navItem__bakjf a')[2:-1]
temp_link = [link.get_attribute('href') for link in temp_link] # Category link

link_prd_len = [5,2,5,4] # number of products for each category : 24.01.01

global_i = 0
b_stopped = False

for i, link in enumerate(temp_link):
  print('')
  print(temp_text[i])

  driver.get(link)

  if temp_text[i] not in ['CLEANSER']:
    selector = 'div.ProductGridItem__imageContainer__Ts5N_.default a'
  else:
    # selector = 'a.ProductShowcase__title__SBCBw'
    selector = 'a.ProductShowcase__rating__Wi83c'

  product_link = [dd.get_attribute('href') for dd in driver.find_elements(By.CSS_SELECTOR, selector)] # product link
  while len(product_link) < link_prd_len[i]:
    product_link = [dd.get_attribute('href') for dd in driver.find_elements(By.CSS_SELECTOR, selector)]

  print(f'link 수:{len(product_link)}')

  for j, prd_link in enumerate(product_link):
    review_dict = {'prd_type':'0','prd_name':'a','title':'d','star':'c','review':'e','date':'b','country':'cc'}

    driver.get(prd_link)
    prd_name = driver.find_elements(By.CSS_SELECTOR, '#productTitle')
    prd_name = prd_name[len(prd_name)-1].text
    print(f'\n{j+1}=> {prd_name}')

    # go to review
    while True:
      try:
        if b_stopped:
          print('restart')
          driver.get(prd_link)
          driver.implicitly_wait(1)
          b_stopped = False

        try:
          review_link = driver.find_elements(By.CSS_SELECTOR,'div.a-row.a-spacing-medium a')[1].get_attribute('href') # US review
          b_skip_title = True
        except:
          review_link = driver.find_elements(By.CSS_SELECTOR,'a[data-hook="see-all-reviews-link-foot"]')[0].get_attribute('href') # non-US review
          b_skip_title = False

        driver.get(review_link)

        rating_info_element = driver.find_element(By.CSS_SELECTOR, 'div[data-hook="cr-filter-info-review-rating-count"]')
        rating_info_text = rating_info_element.text.strip()
        # 정규표현식을 사용하여 숫자 추출
        match = re.search(r'(\d+) total ratings, (\d+) with reviews', rating_info_text)

        if match:
            total_ratings = match.group(1)
            with_reviews = match.group(2)

        print(f'number of reviews: {with_reviews}')

        n_review = int(with_reviews)
        if n_review % 10 > 0:
          n_rep = n_review//10 + 1
        else:
          n_rep = n_review//10

        print('pages: ', end='')

        review_middle_all = []
        for rep in range(n_rep):
          time.sleep(1)

          if b_skip_title: review_title = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-title"]')[2:]]
          else: review_title = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-title"]')]
          review_date = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,"span[data-hook='review-date']")]
          review_country = [re.search(r'Reviewed in (.*?) on (.*)', tt).group(1) for tt in review_date]
          review_date = [re.search(r'Reviewed in (.*?) on (.*)', tt).group(2) for tt in review_date]
          review_star = [tt.get_attribute("textContent") for tt in driver.find_elements(By.CSS_SELECTOR,"i[data-hook='review-star-rating'] span.a-icon-alt")]
          if len(review_star) < len(review_date): # other language
            review_star = review_star + [tt.get_attribute("textContent") for tt in driver.find_elements(By.CSS_SELECTOR,"i[data-hook='cmps-review-star-rating'] span.a-icon-alt")]
          review_star = [re.search(r'(\d+\.\d+) out of \d+ stars', tt).group(1) for tt in review_star]

          review_body = [tt.text for tt in driver.find_elements(By.CSS_SELECTOR,'[data-hook="review-body"]')]
          # print(f'리뷰갯수: {len(review_title),len(review_date), len(review_star), len(review_body)}')

          print(rep+1, end=" ")
          # review_star = driver.execute_script("return document.querySelector('i[data-hook=\"review-star-rating\"] span.a-icon-alt').textContent")
          # print(f'Title: {review_title[0]}')
          # print(review_date[0])
          # print(review_star[0])
          # print(review_body[0])

          for k in range(len(review_body)): # for each review
            review_dict['prd_type'] = temp_text[i]
            review_dict['prd_name'] = prd_name
            review_dict['title'] = review_title[k]
            review_dict['star'] = review_star[k]
            review_dict['review'] = review_body[k]
            review_dict['date'] = review_date[k]
            review_dict['country'] = review_country[k]
            review_middle_all = review_middle_all + [pd.DataFrame([review_dict])]

          if n_review > 10:
            driver.find_elements(By.CSS_SELECTOR,'li.a-last')[0].click() # next button

        review_all_anua = review_all_anua + review_middle_all # for each product

        print('')

        break

      except Exception as e:
        # 해당 제품에서 error 발생시, while문에서 restart
        # print(f"Error: {e}")
        print(f"Error:")
        # account login 요구 시, 해당 부분 실행
        try:
          acnt = pd.read_csv('acnt_inform.csv') # your account information
          email_input = driver.find_element(By.ID, "ap_email")
          email_value = acnt.email.iloc[0]
          email_input.send_keys(email_value)

          continue_button = driver.find_element(By.ID, "continue")
          continue_button.click()

          password_input = driver.find_element(By.ID, "ap_password")
          password_value = acnt.password.iloc[0]
          password_input.send_keys(password_value)

          print('login complete')
        except:
          # 다른 에러의 경우,
          print('not login error')

        b_stopped = True
        global_i = i
        global_j = j

driver.quit()


SERUM
link 수:5

1=> Anua Dark Spot Correcting Serum / 10% Niacinamide+ 4% Tranexamic Acid, for Post-Acne Marks, Acne Scars, Hyperpigmentation and Even Skin Tone, Fragrance-Free (30ml /1.01 fl.oz.)
number of reviews: 10
pages: 1 

2=> Anua Heartleaf 80% Soothing Ampoule 30ml / 1.01 fl.oz. I non-greasy, highly concentrated skin calm serum hydrating panthenol B5 calming treatment essence for combination, sensitive, normal skin, Korea
number of reviews: 61
pages: 1 2 3 4 5 6 7 

3=> Anua Peach 70% Niacinamide Serum 30ml / brightening hydrating face serum hyperpigmentation treatment reducing melanine daily clean beauty (1.01 fl. oz.)
number of reviews: 52
pages: 1 2 3 4 5 6 

4=> Anua Green Lemon Vitamin C Serum with Vitamin E, Hyaluronic & Ferulic Acid 0.67 fl.oz / 20ml
number of reviews: 34
pages: 1 2 3 4 

5=> ANUA BIRCH 70 MOISTURE BOOSTING SERUM 30ml
number of reviews: 28
pages: 1 2 3 

MOISTURIZER
link 수:4

1=> ANUA Heartleaf 70 Intense Calming Cream with Ceramide, Panthenol, Heartle

In [16]:
review_df_anua = pd.concat(review_all_anua).drop_duplicates()
review_df_anua.groupby(['country'])['date'].count()

country
Australia               3
Canada                  2
Germany                 2
Italy                   1
Japan                  53
the United Kingdom      4
the United States     761
Name: date, dtype: int64

In [18]:
review_df_anua.to_csv('../numbuzin/amazon_anua_review.csv',index=None)